<a href="https://colab.research.google.com/github/Mihan0207/Avatar_Fire_and_Ash_Prediction/blob/main/Avatar_Fire%26Ash_Revenue_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction and Library Imports

This initial cell serves two primary purposes:

1.  **Project Overview:** It provides a high-level description of the machine learning project,
    which aims to predict the opening revenue for movies, with a specific focus on
    "Avatar: Fire and Ash". It also lists the various ML models that will be compared
    for this prediction task.

2.  **Environment Setup:** It imports all necessary Python libraries required for
    data manipulation (pandas, numpy, datetime), machine learning (sklearn, xgboost,
    lightgbm, catboost), and data visualization (matplotlib, seaborn).
    Warnings are also configured to be ignored for cleaner output.

This cell prepares the environment and sets the stage for the subsequent data loading,
preprocessing, modeling, and evaluation steps.

In [3]:
!pip install catboost

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, ElasticNet
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

### Step 1: Load and Explore Data

This cell is responsible for loading the initial datasets required for the movie revenue prediction project. It loads three CSV files into pandas DataFrames: `cleaned_movie_data_ml.csv` (main movie data), `dim_director.csv` (director-specific information), and `dim_distributor.csv` (distributor-specific information). After loading, it prints the dimensions (rows and columns) of each DataFrame and lists their respective column names, providing an immediate overview of the available data.

In [7]:
# ============================================================================
# STEP 1: LOAD AND EXPLORE DATA
# ============================================================================

print("=" * 80)
print("STEP 1: LOADING DATA")
print("=" * 80)

# Load all three datasets
movies_df = pd.read_csv('/content/cleaned_movie_data_ml.csv')
directors_df = pd.read_csv('/content/dim_director.csv')
distributors_df = pd.read_csv('/content/dim_distributor.csv')

print(f"\n📊 Movies Dataset: {movies_df.shape[0]} rows, {movies_df.shape[1]} columns")
print(f"📊 Directors Dataset: {directors_df.shape[0]} rows, {directors_df.shape[1]} columns")
print(f"📊 Distributors Dataset: {distributors_df.shape[0]} rows, {distributors_df.shape[1]} columns")

# Display column names
print("\n🔹 Movies Columns:", list(movies_df.columns))
print("\n🔹 Directors Columns:", list(directors_df.columns))
print("\n🔹 Distributors Columns:", list(distributors_df.columns))

STEP 1: LOADING DATA

📊 Movies Dataset: 3332 rows, 23 columns
📊 Directors Dataset: 1656 rows, 7 columns
📊 Distributors Dataset: 186 rows, 7 columns

🔹 Movies Columns: ['title', 'year', 'release_date', 'primary_genre', 'total_gross', 'opening_revenue', 'final_budget', 'distributor', 'tmdb_id', 'imdb_id', 'tmdb_rating', 'imdb_rating', 'tmdb_votes', 'runtime', 'director', 'genres', 'genre_Drama', 'genre_Comedy', 'genre_Action', 'genre_Adventure', 'genre_Thriller', 'genre_Science Fiction', 'genre_Other']

🔹 Directors Columns: ['director_name', 'total_movies', 'avg_total_gross', 'avg_opening_revenue', 'avg_final_budget', 'avg_tmdb_rating', 'director_id']

🔹 Distributors Columns: ['distributor_name', 'total_movies', 'total_gross', 'avg_opening_revenue', 'avg_final_budget', 'market_share', 'distributor_id']


### Step 2: Data Preprocessing and Feature Engineering

This cell performs critical steps to prepare the raw data for machine learning models. It involves:

1.  **Data Copying:** Creates a working copy of the `movies_df` to ensure the original DataFrame remains untouched.
2.  **Date Parsing & Temporal Feature Extraction:** Converts the `release_date` column into a datetime object and extracts various temporal features such as `release_month`, `release_day`, `release_dayofweek`, `release_quarter`, and `release_weekofyear`. It also generates binary indicators for `is_holiday_release`, `is_summer_release`, and `is_weekend_release`.
3.  **Merging Director Information:** Integrates director-specific aggregated data from `directors_df` into the main DataFrame, adding features like `director_total_movies`, `director_avg_opening_revenue`, `director_avg_budget`, etc.
4.  **Merging Distributor Information:** Incorporates distributor-specific aggregated data from `distributors_df`, bringing in features such as `distributor_total_movies`, `distributor_avg_opening_revenue`, and `distributor_market_share`.
5.  **Creation of Additional Features:** Derives new features to enhance predictive power:
    *   `budget_per_rating`: Ratio of `final_budget` to `tmdb_rating`.
    *   `genre_count`: Number of genres associated with a movie.
    *   `is_major_studio`: A binary flag indicating if the distributor is one of the predefined major studios.
    *   `director_experience`: Categorizes directors based on their total number of movies.
6.  **Missing Value Handling:** Identifies and imputes missing numerical values using the median. Categorical IDs (`director_id`, `distributor_id`) are filled with -1, and `director_experience` with 1, to ensure data completeness for modeling.

In [8]:
# ============================================================================
# STEP 2: DATA PREPROCESSING AND FEATURE ENGINEERING
# ============================================================================

print("\n" + "=" * 80)
print("STEP 2: DATA PREPROCESSING & FEATURE ENGINEERING")
print("=" * 80)

# 2.1 Create a copy for processing
df = movies_df.copy()

# 2.2 Parse release date and extract temporal features
def parse_date(date_str):
    """Parse various date formats"""
    try:
        # Try MM/DD/YY format
        return pd.to_datetime(date_str, format='%m/%d/%y')
    except:
        try:
            # Try other formats
            return pd.to_datetime(date_str)
        except:
            return pd.NaT

df['release_date_parsed'] = df['release_date'].apply(parse_date)

# Extract temporal features
df['release_month'] = df['release_date_parsed'].dt.month
df['release_day'] = df['release_date_parsed'].dt.day
df['release_dayofweek'] = df['release_date_parsed'].dt.dayofweek  # 0=Monday, 6=Sunday
df['release_quarter'] = df['release_date_parsed'].dt.quarter
df['release_weekofyear'] = df['release_date_parsed'].dt.isocalendar().week.astype('Int64')

# Is it a holiday season release? (Nov-Dec or Summer Jun-Aug)
df['is_holiday_release'] = df['release_month'].isin([11, 12]).astype(int)
df['is_summer_release'] = df['release_month'].isin([6, 7, 8]).astype(int)
df['is_weekend_release'] = df['release_dayofweek'].isin([4, 5, 6]).astype(int)  # Fri, Sat, Sun

print("✅ Temporal features extracted")

# 2.3 Merge with Director information
directors_df_renamed = directors_df.rename(columns={
    'director_name': 'director',
    'total_movies': 'director_total_movies',
    'avg_total_gross': 'director_avg_total_gross',
    'avg_opening_revenue': 'director_avg_opening_revenue',
    'avg_final_budget': 'director_avg_budget',
    'avg_tmdb_rating': 'director_avg_rating'
})

df = df.merge(directors_df_renamed[['director', 'director_id', 'director_total_movies',
                                     'director_avg_total_gross', 'director_avg_opening_revenue',
                                     'director_avg_budget', 'director_avg_rating']],
              on='director', how='left')

print("✅ Director features merged")

# 2.4 Merge with Distributor information
distributors_df_renamed = distributors_df.rename(columns={
    'distributor_name': 'distributor',
    'total_movies': 'distributor_total_movies',
    'total_gross': 'distributor_total_gross',
    'avg_opening_revenue': 'distributor_avg_opening_revenue',
    'avg_final_budget': 'distributor_avg_budget',
    'market_share': 'distributor_market_share'
})

df = df.merge(distributors_df_renamed[['distributor', 'distributor_id', 'distributor_total_movies',
                                        'distributor_total_gross', 'distributor_avg_opening_revenue',
                                        'distributor_avg_budget', 'distributor_market_share']],
              on='distributor', how='left')

print("✅ Distributor features merged")

# 2.5 Create additional features
# Budget-to-rating ratio
df['budget_per_rating'] = df['final_budget'] / (df['tmdb_rating'] + 0.1)

# Genre counts
genre_cols = [col for col in df.columns if col.startswith('genre_')]
df['genre_count'] = df[genre_cols].sum(axis=1)

# Is it a major studio?
major_studios = ['Warner Bros.', 'Paramount Pictures', 'Sony Pictures Releasing',
                 'Walt Disney Studios Motion Pictures', 'Universal Pictures',
                 'Twentieth Century Fox', '20th Century Studios', 'Lionsgate']
df['is_major_studio'] = df['distributor'].isin(major_studios).astype(int)

# Director experience level
df['director_experience'] = pd.cut(df['director_total_movies'].fillna(1),
                                   bins=[0, 2, 5, 10, 100],
                                   labels=[1, 2, 3, 4]).astype(float)

print("✅ Additional features created")

# 2.6 Handle missing values
print(f"\n📊 Missing values before imputation:")
missing_cols = df.isnull().sum()
print(missing_cols[missing_cols > 0])

# Fill missing values with median for numerical columns
numerical_cols_to_fill = ['director_total_movies', 'director_avg_total_gross',
                          'director_avg_opening_revenue', 'director_avg_budget',
                          'director_avg_rating', 'distributor_total_movies',
                          'distributor_total_gross', 'distributor_avg_opening_revenue',
                          'distributor_avg_budget', 'distributor_market_share',
                          'release_month', 'release_day', 'release_dayofweek',
                          'release_quarter', 'release_weekofyear', 'final_budget',
                          'runtime', 'tmdb_rating', 'tmdb_votes']

for col in numerical_cols_to_fill:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Fill categorical with mode
df['director_id'] = df['director_id'].fillna(-1)
df['distributor_id'] = df['distributor_id'].fillna(-1)
df['director_experience'] = df['director_experience'].fillna(1)

print("✅ Missing values handled")


STEP 2: DATA PREPROCESSING & FEATURE ENGINEERING
✅ Temporal features extracted
✅ Director features merged
✅ Distributor features merged
✅ Additional features created

📊 Missing values before imputation:
distributor                        17
imdb_id                            10
imdb_rating                        12
director                            8
genres                              3
director_id                         8
director_total_movies               8
director_avg_total_gross            8
director_avg_opening_revenue        8
director_avg_budget                 8
director_avg_rating                 8
distributor_id                     17
distributor_total_movies           17
distributor_total_gross            17
distributor_avg_opening_revenue    17
distributor_avg_budget             17
distributor_market_share           17
dtype: int64
✅ Missing values handled


### Step 3: Prepare Features for Modeling

This cell defines and prepares the feature set (`X`) and the target variable (`y`) for the machine learning models. It performs the following actions:

1.  **Feature Column Selection:** A comprehensive list of `feature_columns` is explicitly defined, including core movie details, temporal attributes, genre indicators, and the merged director and distributor statistics, along with the newly engineered features.
2.  **Target Variable Definition:** The `opening_revenue` column is designated as the `target` variable.
3.  **Data Filtering:** The DataFrame is filtered to include only rows where the `opening_revenue` is not missing and is greater than zero, ensuring a clean target for prediction.
4.  **Feature Validation:** Checks if all specified `feature_columns` are present in the filtered DataFrame and reports any discrepancies.
5.  **Dataset Separation:** The filtered DataFrame is split into `X` (features) and `y` (target).
6.  **Final NaN Handling:** Any remaining `NaN` values in the feature matrix `X` are filled with `0`.
7.  **Target Statistics Display:** Basic statistical summaries (mean, median, standard deviation, min, max) of the `opening_revenue` are printed to understand its distribution.

In [9]:
# ============================================================================
# STEP 3: PREPARE FEATURES FOR MODELING
# ============================================================================

print("\n" + "=" * 80)
print("STEP 3: PREPARE FEATURES FOR MODELING")
print("=" * 80)

# Select features for modeling
feature_columns = [
    # Core movie features
    'year', 'final_budget', 'tmdb_rating', 'tmdb_votes', 'runtime',

    # Temporal features
    'release_month', 'release_day', 'release_dayofweek', 'release_quarter',
    'is_holiday_release', 'is_summer_release', 'is_weekend_release',

    # Genre features
    'genre_Drama', 'genre_Comedy', 'genre_Action', 'genre_Adventure',
    'genre_Thriller', 'genre_Science Fiction', 'genre_Other', 'genre_count',

    # Director features
    'director_id', 'director_total_movies', 'director_avg_total_gross',
    'director_avg_opening_revenue', 'director_avg_budget', 'director_avg_rating',
    'director_experience',

    # Distributor features
    'distributor_id', 'distributor_total_movies', 'distributor_total_gross',
    'distributor_avg_opening_revenue', 'distributor_avg_budget',
    'distributor_market_share', 'is_major_studio',

    # Engineered features
    'budget_per_rating'
]

# Target variable
target = 'opening_revenue'

# Filter rows with valid target
df_model = df[df[target].notna() & (df[target] > 0)].copy()

# Verify all feature columns exist
available_features = [col for col in feature_columns if col in df_model.columns]
missing_features = [col for col in feature_columns if col not in df_model.columns]

if missing_features:
    print(f"⚠️ Missing features (will be skipped): {missing_features}")

print(f"✅ Using {len(available_features)} features for modeling")

# Prepare X and y
X = df_model[available_features].copy()
y = df_model[target].copy()

# Handle any remaining NaN values
X = X.fillna(0)

print(f"\n📊 Dataset shape: X={X.shape}, y={y.shape}")
print(f"📊 Target (opening_revenue) statistics:")
print(f"   Mean: ${y.mean():,.0f}")
print(f"   Median: ${y.median():,.0f}")
print(f"   Std: ${y.std():,.0f}")
print(f"   Min: ${y.min():,.0f}")
print(f"   Max: ${y.max():,.0f}")



STEP 3: PREPARE FEATURES FOR MODELING
✅ Using 35 features for modeling

📊 Dataset shape: X=(3332, 35), y=(3332,)
📊 Target (opening_revenue) statistics:
   Mean: $21,262,104
   Median: $7,866,504
   Std: $39,883,720
   Min: $962
   Max: $473,894,638


### Log Transformation for Skewed Target

This is a crucial preprocessing step for the target variable, `opening_revenue`, which typically exhibits a highly skewed distribution (many low values, few very high values). Applying a logarithmic transformation helps to:

*   **Reduce Skewness:** Makes the distribution more symmetric, bringing it closer to a normal distribution, which is often an assumption for many regression models.
*   **Stabilize Variance:** Can help in making the variance more consistent across the range of the target variable.
*   **Improve Model Performance:** Models often perform better when trained on transformed targets with more normalized distributions.

The `np.log1p(y)` function is used, which calculates `log(1 + y)`. This is preferred over `log(y)` because it gracefully handles zero values (where `log(0)` is undefined) and behaves similarly to `log(y)` for larger values. The cell outputs the skewness before and after the transformation to demonstrate its effect.

In [10]:
# ============================================================================
# CRITICAL: LOG TRANSFORMATION FOR SKEWED TARGET
# ============================================================================

print("\n" + "=" * 80)
print("LOG TRANSFORMATION (CRITICAL FOR REVENUE DATA)")
print("=" * 80)

# Apply log1p transformation to handle skewed revenue distribution
y_log = np.log1p(y)

print(f"📊 Original Target - Skewness: {y.skew():.2f}")
print(f"📊 Log-transformed Target - Skewness: {y_log.skew():.2f}")
print("✅ Log transformation applied to reduce skewness and improve model performance")


LOG TRANSFORMATION (CRITICAL FOR REVENUE DATA)
📊 Original Target - Skewness: 4.35
📊 Log-transformed Target - Skewness: -0.53
✅ Log transformation applied to reduce skewness and improve model performance


### Step 4: Train-Test Split

This cell divides the dataset into training and testing sets, a standard practice to evaluate a machine learning model's performance on unseen data. Here's a breakdown:

1.  **Splitting Data:** The `train_test_split` function from `sklearn.model_selection` is used to partition the features (`X`) and the log-transformed target (`y_log`) into training (`X_train`, `y_train_log`) and testing (`X_test`, `y_test_log`) subsets. A `test_size` of `0.2` means 20% of the data is reserved for testing, and `random_state=42` ensures that the split is reproducible.
2.  **Original Scale Target Variables:** Separate variables `y_train_orig` and `y_test_orig` are created by applying `np.expm1` (the inverse of `np.log1p`) to the log-transformed target variables. These original-scale target variables are essential for calculating evaluation metrics (like MAE and RMSE) in terms of actual dollar amounts, making them directly interpretable.

The cell concludes by printing the number of samples in both the training and test sets.

In [11]:
# ============================================================================
# STEP 4: TRAIN-TEST SPLIT
# ============================================================================

print("\n" + "=" * 80)
print("STEP 4: TRAIN-TEST SPLIT")
print("=" * 80)

# Split using log-transformed target
X_train, X_test, y_train_log, y_test_log = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

# Keep original scale for evaluation
y_train_orig = np.expm1(y_train_log)
y_test_orig = np.expm1(y_test_log)

print(f"✅ Training set: {X_train.shape[0]} samples")
print(f"✅ Test set: {X_test.shape[0]} samples")


STEP 4: TRAIN-TEST SPLIT
✅ Training set: 2665 samples
✅ Test set: 667 samples


### Step 5: Define and Compare Multiple Models

This extensive cell defines, trains, and evaluates several machine learning models, including an ensemble stacking model, to find the best performer for predicting movie opening revenue.

1.  **`evaluate_model_detailed` Function:** A helper function is defined to streamline the evaluation process. It trains a given model on the log-transformed training data, makes predictions, and then converts both actuals and predictions back to the original scale to calculate key metrics: RMSE, MAE, R², and MAPE (Mean Absolute Percentage Error).
2.  **Individual Model Definitions:** Five different regression models are instantiated with pre-tuned hyperparameters:
    *   **XGBoost (`xgb.XGBRegressor`):** A highly efficient and robust gradient boosting algorithm.
    *   **LightGBM (`lgb.LGBMRegressor`):** Another fast, distributed, high-performance gradient boosting framework.
    *   **CatBoost (`CatBoostRegressor`):** A gradient boosting library known for its handling of categorical features and robustness.
    *   **Random Forest (`RandomForestRegressor`):** An ensemble method that operates by constructing multiple decision trees during training.
    *   **Gradient Boosting (`GradientBoostingRegressor`):** A powerful ensemble technique that builds models sequentially, each one trying to correct the errors of the previous one.
3.  **Model Training and Evaluation Loop:** Each model is trained on the `X_train` and `y_train_log` data, and its performance is evaluated on the test set. The `Test R²` and `Test MAE` are printed for each model.

### Step 5.5: Create Ensemble (Stacking) Model

1.  **Base Estimators:** A set of `base_estimators` (XGBoost, LightGBM, CatBoost, Random Forest) is defined. These models will form the first layer of the stacking ensemble.
2.  **Stacking Regressor:** A `StackingRegressor` is created, which combines the predictions of the `base_estimators` using a `Ridge` regression as the `final_estimator` (meta-learner). This approach leverages the strengths of multiple models.
3.  **Ensemble Training:** The stacking model is then trained and evaluated, and its performance metrics are added to the results.

In [12]:
# ============================================================================
# STEP 5: DEFINE AND COMPARE MULTIPLE MODELS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 5: COMPARING MULTIPLE ML MODELS")
print("=" * 80)

# Define evaluation function
def evaluate_model_detailed(model, X_train, X_test, y_train_log, y_test_log, model_name):
    """Train and evaluate model with both log and original scale metrics"""

    # Train
    model.fit(X_train, y_train_log)

    # Predict (log scale)
    y_pred_train_log = model.predict(X_train)
    y_pred_test_log = model.predict(X_test)

    # Convert back to original scale
    y_pred_train = np.expm1(y_pred_train_log)
    y_pred_test = np.expm1(y_pred_test_log)
    y_train_orig = np.expm1(y_train_log)
    y_test_orig = np.expm1(y_test_log)

    # Calculate metrics on original scale
    train_rmse = np.sqrt(mean_squared_error(y_train_orig, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_test))
    train_mae = mean_absolute_error(y_train_orig, y_pred_train)
    test_mae = mean_absolute_error(y_test_orig, y_pred_test)
    train_r2 = r2_score(y_train_orig, y_pred_train)
    test_r2 = r2_score(y_test_orig, y_pred_test)

    # MAPE (avoiding division by zero)
    train_mape = np.mean(np.abs((y_train_orig - y_pred_train) / (y_train_orig + 1))) * 100
    test_mape = np.mean(np.abs((y_test_orig - y_pred_test) / (y_test_orig + 1))) * 100

    return {
        'model_name': model_name,
        'model': model,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_mape': train_mape,
        'test_mape': test_mape,
        'y_pred_test': y_pred_test,
        'y_pred_test_log': y_pred_test_log
    }

# Define models with optimized parameters (less overfitting)
models = {
    'XGBoost': xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=300,
        max_depth=4,  # Reduced from 6
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        min_child_weight=5,
        gamma=0.2,
        reg_alpha=0.5,
        reg_lambda=2.0,
        random_state=42,
        n_jobs=-1
    ),

    'LightGBM': lgb.LGBMRegressor(
        objective='regression',
        n_estimators=300,
        max_depth=4,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        min_child_samples=20,
        reg_alpha=0.5,
        reg_lambda=2.0,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),

    'CatBoost': CatBoostRegressor(
        iterations=300,
        depth=4,
        learning_rate=0.03,
        l2_leaf_reg=5,
        random_seed=42,
        verbose=0
    ),

    'Random Forest': RandomForestRegressor(
        n_estimators=300,
        max_depth=8,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),

    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.03,
        subsample=0.7,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    )
}

# Train and evaluate all models
print("\n🔄 Training and evaluating models...")
results = {}

for name, model in models.items():
    print(f"\n   Training {name}...")
    results[name] = evaluate_model_detailed(
        model, X_train, X_test, y_train_log, y_test_log, name
    )
    print(f"   ✅ {name}: Test R² = {results[name]['test_r2']:.4f}, Test MAE = ${results[name]['test_mae']:,.0f}")

# ============================================================================
# STEP 5.5: CREATE ENSEMBLE (STACKING) MODEL
# ============================================================================

print("\n" + "=" * 80)
print("STEP 5.5: BUILDING ENSEMBLE (STACKING) MODEL")
print("=" * 80)

# Define base estimators for stacking
base_estimators = [
    ('xgb', xgb.XGBRegressor(
        n_estimators=200, max_depth=4, learning_rate=0.03,
        subsample=0.7, colsample_bytree=0.7, random_state=42, n_jobs=-1
    )),
    ('lgb', lgb.LGBMRegressor(
        n_estimators=200, max_depth=4, learning_rate=0.03,
        subsample=0.7, colsample_bytree=0.7, random_state=42, n_jobs=-1, verbose=-1
    )),
    ('cat', CatBoostRegressor(
        iterations=200, depth=4, learning_rate=0.03,
        random_seed=42, verbose=0
    )),
    ('rf', RandomForestRegressor(
        n_estimators=200, max_depth=6, min_samples_leaf=5,
        random_state=42, n_jobs=-1
    ))
]

# Create stacking regressor with Ridge as final estimator
stacking_model = StackingRegressor(
    estimators=base_estimators,
    final_estimator=Ridge(alpha=1.0),
    cv=5,
    n_jobs=-1
)

print("🔄 Training Stacking Ensemble (this may take a moment)...")
results['Stacking Ensemble'] = evaluate_model_detailed(
    stacking_model, X_train, X_test, y_train_log, y_test_log, 'Stacking Ensemble'
)
print(f"✅ Stacking Ensemble: Test R² = {results['Stacking Ensemble']['test_r2']:.4f}, Test MAE = ${results['Stacking Ensemble']['test_mae']:,.0f}")



STEP 5: COMPARING MULTIPLE ML MODELS

🔄 Training and evaluating models...

   Training XGBoost...
   ✅ XGBoost: Test R² = 0.6879, Test MAE = $8,852,665

   Training LightGBM...
   ✅ LightGBM: Test R² = 0.6906, Test MAE = $8,996,675

   Training CatBoost...
   ✅ CatBoost: Test R² = 0.5536, Test MAE = $10,272,286

   Training Random Forest...
   ✅ Random Forest: Test R² = 0.5398, Test MAE = $10,815,545

   Training Gradient Boosting...
   ✅ Gradient Boosting: Test R² = 0.7286, Test MAE = $8,350,308

STEP 5.5: BUILDING ENSEMBLE (STACKING) MODEL
🔄 Training Stacking Ensemble (this may take a moment)...
✅ Stacking Ensemble: Test R² = 0.7133, Test MAE = $8,712,309


### Step 6: Model Comparison and Selection

This cell compares the performance of all trained models and selects the single best model based on its test set metrics.

1.  **Comparison DataFrame:** A pandas DataFrame, `comparison_df`, is created to aggregate the performance metrics (Train R², Test R², Train MAE, Test MAE, Test MAPE, and an Overfit Ratio) for all models.
2.  **Sorted Output:** The `comparison_df` is sorted by `Test R²` in descending order and printed in a clean, formatted table. This allows for a quick and clear comparison of how well each model generalizes to unseen data.
3.  **Best Model Identification:** The model with the highest `Test R²` score is identified as the `best_model`. Its name, the model object itself, and its performance metrics are stored in `best_model_name`, `best_model`, and `best_metrics` variables respectively.
4.  **Cross-Validation:** To further validate the `best_model`'s robustness, 5-fold cross-validation is performed on the entire dataset (`X` and `y_log`). The mean and standard deviation of the cross-validation R² scores are printed, providing a more reliable estimate of the model's generalization ability.
5.  **Storing Test Metrics:** The test predictions (`y_pred_test`) from the best model and its key performance metrics (`test_metrics`) are stored for use in subsequent steps (e.g., visualizations and final summary).

In [13]:
# ============================================================================
# STEP 6: MODEL COMPARISON AND SELECTION
# ============================================================================

print("\n" + "=" * 80)
print("STEP 6: MODEL COMPARISON AND EVALUATION")
print("=" * 80)

# Create comparison dataframe
comparison_df = pd.DataFrame([
    {
        'Model': name,
        'Train R²': res['train_r2'],
        'Test R²': res['test_r2'],
        'Train MAE': res['train_mae'],
        'Test MAE': res['test_mae'],
        'Train RMSE': res['train_rmse'],
        'Test RMSE': res['test_rmse'],
        'Train MAPE': res['train_mape'],
        'Test MAPE': res['test_mape'],
        'Overfit Ratio': res['train_r2'] / max(res['test_r2'], 0.001)
    }
    for name, res in results.items()
]).sort_values('Test R²', ascending=False)

print("\n📊 MODEL COMPARISON (Sorted by Test R²):")
print("=" * 120)
print(f"{'Model':<20} {'Train R²':>10} {'Test R²':>10} {'Train MAE':>15} {'Test MAE':>15} {'Test MAPE':>12} {'Overfit':>10}")
print("-" * 120)

for _, row in comparison_df.iterrows():
    print(f"{row['Model']:<20} {row['Train R²']:>10.4f} {row['Test R²']:>10.4f} "
          f"${row['Train MAE']:>13,.0f} ${row['Test MAE']:>13,.0f} "
          f"{row['Test MAPE']:>10.2f}% {row['Overfit Ratio']:>9.2f}x")

# Select best model based on Test R² (or you can use Test MAE)
best_model_name = comparison_df.iloc[0]['Model']
best_model = results[best_model_name]['model']
best_metrics = results[best_model_name]

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   • Test R²: {best_metrics['test_r2']:.4f}")
print(f"   • Test MAE: ${best_metrics['test_mae']:,.0f}")
print(f"   • Test RMSE: ${best_metrics['test_rmse']:,.0f}")
print(f"   • Test MAPE: {best_metrics['test_mape']:.2f}%")

# Cross-validation for best model
print(f"\n📊 5-Fold Cross-Validation for {best_model_name}:")
cv_scores = cross_val_score(best_model, X, y_log, cv=5, scoring='r2')
print(f"   CV R² Scores: {cv_scores}")
print(f"   Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# Store test metrics for later use
y_pred_test = best_metrics['y_pred_test']
test_metrics = {
    'r2': best_metrics['test_r2'],
    'mae': best_metrics['test_mae'],
    'rmse': best_metrics['test_rmse'],
    'mape': best_metrics['test_mape']
}



STEP 6: MODEL COMPARISON AND EVALUATION

📊 MODEL COMPARISON (Sorted by Test R²):
Model                  Train R²    Test R²       Train MAE        Test MAE    Test MAPE    Overfit
------------------------------------------------------------------------------------------------------------------------
Gradient Boosting        0.8177     0.7286 $    6,615,083 $    8,350,308     290.68%      1.12x
Stacking Ensemble        0.7557     0.7133 $    7,367,392 $    8,712,309     301.45%      1.06x
LightGBM                 0.7421     0.6906 $    7,427,679 $    8,996,675     299.34%      1.07x
XGBoost                  0.7577     0.6879 $    7,238,218 $    8,852,665     302.92%      1.10x
CatBoost                 0.5686     0.5536 $    9,727,191 $   10,272,286     386.74%      1.03x
Random Forest            0.5953     0.5398 $    9,550,550 $   10,815,545     325.91%      1.10x

🏆 BEST MODEL: Gradient Boosting
   • Test R²: 0.7286
   • Test MAE: $8,350,308
   • Test RMSE: $20,702,441
   • Test MAPE

### Step 7: Feature Importance Analysis

This cell identifies and visualizes the most influential features that the `best_model` used to make its predictions. Understanding feature importance can provide valuable insights into the data and the model's decision-making process.

1.  **Retrieve Importances:** The code dynamically retrieves feature importances from the `best_model`. For tree-based models (like Gradient Boosting, XGBoost), it directly accesses the `feature_importances_` attribute. If the `best_model` is a `StackingRegressor` (which doesn't have a single `feature_importances_`), it defaults to using the importances from the XGBoost model within the ensemble, as XGBoost usually provides good feature importance scores.
2.  **Create DataFrame:** The feature names and their corresponding importance scores are compiled into a pandas DataFrame and sorted in descending order of importance.
3.  **Display Top Features:** The top 15 most important features are printed to the console.
4.  **Visualize Importances:** A horizontal bar chart is generated to graphically represent the top 20 features and their importance scores. This plot is saved as `feature_importance.png`, offering a clear visual summary of which features contributed most to the model's predictions.

In [14]:
# ============================================================================
# STEP 7: FEATURE IMPORTANCE ANALYSIS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 7: FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

# Get feature importance (handle different model types)
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
elif hasattr(best_model, 'estimators_'):
    # For stacking, use XGBoost importance
    importances = results['XGBoost']['model'].feature_importances_
else:
    importances = np.zeros(len(available_features))

feature_importance = pd.DataFrame({
    'feature': available_features,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\n🔝 Top 15 Most Important Features:")
print(feature_importance.head(15).to_string(index=False))

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(20)
plt.barh(range(len(top_features)), top_features['importance'].values)
plt.yticks(range(len(top_features)), top_features['feature'].values)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title(f'Top 20 Feature Importances - {best_model_name}')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.close()
print("\n✅ Feature importance plot saved as 'feature_importance.png'")



STEP 7: FEATURE IMPORTANCE ANALYSIS

🔝 Top 15 Most Important Features:
                        feature  importance
   director_avg_opening_revenue    0.587761
distributor_avg_opening_revenue    0.177599
              budget_per_rating    0.031471
            director_avg_budget    0.026602
                     tmdb_votes    0.023803
                   final_budget    0.022849
                  release_month    0.013410
                    director_id    0.013108
          director_total_movies    0.012872
              release_dayofweek    0.012706
                           year    0.012303
             is_weekend_release    0.008308
                    tmdb_rating    0.007767
       director_avg_total_gross    0.007458
            director_avg_rating    0.007012

✅ Feature importance plot saved as 'feature_importance.png'


### Step 8: Prediction for 'Avatar: Fire and Ash'

This pivotal cell uses the trained `best_model` to predict the opening revenue for the highly anticipated movie 'Avatar: Fire and Ash' and provides extensive context for this prediction.

1.  **Define Hypothetical Movie Features:** A dictionary `avatar_features` is created to encapsulate all relevant information for 'Avatar: Fire and Ash', including its estimated budget, release date, director, distributor, and genre.
2.  **Retrieve Historical Stats:** It dynamically fetches historical performance data for the director (James Cameron) and distributor (Walt Disney Studios Motion Pictures) from the pre-loaded dimension tables (`directors_df`, `distributors_df`). These aggregated statistics are crucial inputs for the model.
3.  **Construct Feature Vector:** A `avatar_feature_vector` is carefully assembled. This dictionary contains all the features needed by the model, matching the `available_features` used during training. It includes temporal features derived from the release date and engineered features like `budget_per_rating`.
4.  **Create Prediction DataFrame:** A pandas DataFrame, `avatar_df`, is created from the feature vector, ensuring its structure (columns and order) is identical to the training data.
5.  **Generate Prediction:** The `best_model` makes a prediction on `avatar_df`. The output is initially in log scale (`avatar_pred_log`) and is then transformed back to the original dollar amount using `np.expm1` to get `avatar_prediction`.
6.  **Ensemble Predictions:** Predictions from all individual models (XGBoost, LightGBM, CatBoost, Random Forest, Gradient Boosting, and Stacking Ensemble) are also generated and displayed. An `ensemble_avg` is calculated, offering a potentially more robust prediction by combining multiple model outputs.
7.  **Display Results and Context:** The predicted opening revenue from the `best_model` and the `ensemble_avg` are prominently shown. Additional context, such as the release date, budget, and the average historical performance of the director and distributor, is provided.
8.  **Franchise Comparison:** The prediction is benchmarked against the historical opening revenues of previous 'Avatar' movies found in the original `movies_df`.
9.  **95% Prediction Interval:** A 95% prediction interval is calculated based on the standard deviation of the residuals from the test set. This interval provides a range of values within which the actual opening revenue is likely to fall, giving an estimate of the prediction's uncertainty.

In [15]:
# ============================================================================
# STEP 8: PREDICTION FOR AVATAR: FIRE AND ASH
# ============================================================================

print("\n" + "=" * 80)
print("STEP 8: PREDICTING OPENING REVENUE FOR 'AVATAR: FIRE AND ASH'")
print("=" * 80)

# Avatar: Fire and Ash features
avatar_features = {
    'title': 'Avatar: Fire and Ash',
    'year': 2025,
    'release_date': '2025-12-19',  # Friday before Christmas
    'budget': 350_000_000,
    'is_franchise': 1,
    'franchise_name': 'Avatar Collection',
    'director': 'James Cameron',
    'distributor': 'Walt Disney Studios Motion Pictures',
    'genres': 'Action, Adventure, Science Fiction',
    'original_language': 'en',
    'popularity': 500,
    'tmdb_rating': 8.0,
    'tmdb_votes': 50000,
    'vote_average': 8.0,
    'vote_count': 50000,
    'rating': 'PG-13',
    'total_gross': 700_000_000,
    'opening_week': 180_000_000,
    'runtime': 190  # Estimated runtime similar to Way of Water
}

# Get James Cameron's director stats
james_cameron_stats = directors_df[directors_df['director_name'] == 'James Cameron'].iloc[0]
print(f"\n🎬 James Cameron Stats:")
print(f"   Total Movies: {james_cameron_stats['total_movies']}")
print(f"   Avg Total Gross: ${james_cameron_stats['avg_total_gross']:,.0f}")
print(f"   Avg Opening Revenue: ${james_cameron_stats['avg_opening_revenue']:,.0f}")
print(f"   Avg Budget: ${james_cameron_stats['avg_final_budget']:,.0f}")
print(f"   Avg TMDB Rating: {james_cameron_stats['avg_tmdb_rating']:.2f}")

# Get Walt Disney distributor stats
disney_stats = distributors_df[distributors_df['distributor_name'] == 'Walt Disney Studios Motion Pictures'].iloc[0]
print(f"\n🏢 Walt Disney Studios Stats:")
print(f"   Total Movies: {disney_stats['total_movies']}")
print(f"   Total Gross: ${disney_stats['total_gross']:,.0f}")
print(f"   Avg Opening Revenue: ${disney_stats['avg_opening_revenue']:,.0f}")
print(f"   Market Share: {disney_stats['market_share']}%")

# Parse release date for Avatar
avatar_release_date = datetime.strptime(avatar_features['release_date'], '%Y-%m-%d')

# Prepare feature vector for Avatar
avatar_feature_vector = {
    # Core features
    'year': 2025,
    'final_budget': 350_000_000,
    'tmdb_rating': 8.0,
    'tmdb_votes': 50000,
    'runtime': 190,

    # Temporal features
    'release_month': 12,
    'release_day': 19,
    'release_dayofweek': 4,  # Friday
    'release_quarter': 4,
    'is_holiday_release': 1,
    'is_summer_release': 0,
    'is_weekend_release': 1,

    # Genre features (Action, Adventure, Science Fiction)
    'genre_Drama': 0,
    'genre_Comedy': 0,
    'genre_Action': 1,
    'genre_Adventure': 1,
    'genre_Thriller': 0,
    'genre_Science Fiction': 1,
    'genre_Other': 0,
    'genre_count': 3,

    # Director features (James Cameron)
    'director_id': james_cameron_stats['director_id'],
    'director_total_movies': james_cameron_stats['total_movies'],
    'director_avg_total_gross': james_cameron_stats['avg_total_gross'],
    'director_avg_opening_revenue': james_cameron_stats['avg_opening_revenue'],
    'director_avg_budget': james_cameron_stats['avg_final_budget'],
    'director_avg_rating': james_cameron_stats['avg_tmdb_rating'],
    'director_experience': 4,  # 10+ movies = experience level 4

    # Distributor features (Walt Disney)
    'distributor_id': disney_stats['distributor_id'],
    'distributor_total_movies': disney_stats['total_movies'],
    'distributor_total_gross': disney_stats['total_gross'],
    'distributor_avg_opening_revenue': disney_stats['avg_opening_revenue'],
    'distributor_avg_budget': disney_stats['avg_final_budget'],
    'distributor_market_share': disney_stats['market_share'],
    'is_major_studio': 1,

    # Engineered features
    'budget_per_rating': 350_000_000 / 8.1
}

# Create DataFrame with same columns as training data
avatar_df = pd.DataFrame([avatar_feature_vector])

# Ensure column order matches
avatar_df = avatar_df[available_features]

# Make prediction using log scale then convert back
avatar_pred_log = best_model.predict(avatar_df)[0]
avatar_prediction = np.expm1(avatar_pred_log)

# Also get predictions from all models for comparison
print("\n📊 Predictions from all models:")
all_predictions = {}
for name, res in results.items():
    pred_log = res['model'].predict(avatar_df)[0]
    pred = np.expm1(pred_log)
    all_predictions[name] = pred
    print(f"   • {name}: ${pred:,.0f}")

# Use ensemble average for more robust prediction
ensemble_avg = np.mean(list(all_predictions.values()))
print(f"\n   🎯 Ensemble Average: ${ensemble_avg:,.0f}")

print("\n" + "=" * 80)
print("🎬 PREDICTION RESULTS FOR 'AVATAR: FIRE AND ASH'")
print("=" * 80)

print(f"\n🎯 Predicted Opening Revenue ({best_model_name}): ${avatar_prediction:,.0f}")
print(f"🎯 Ensemble Average Prediction: ${ensemble_avg:,.0f}")
print(f"\n📊 Prediction Context:")
print(f"   • Release Date: December 19, 2025 (Friday, Holiday Season)")
print(f"   • Budget: $350,000,000")
print(f"   • Director: James Cameron (Avg Opening: ${james_cameron_stats['avg_opening_revenue']:,.0f})")
print(f"   • Distributor: Walt Disney Studios (Avg Opening: ${disney_stats['avg_opening_revenue']:,.0f})")

# Compare with Avatar franchise history
print("\n📈 Avatar Franchise Comparison:")
avatar_movies = movies_df[movies_df['title'].str.contains('Avatar', case=False, na=False)]
for _, movie in avatar_movies.iterrows():
    print(f"   • {movie['title']} ({movie['year']}): Opening = ${movie['opening_revenue']:,.0f}")

# Prediction intervals (using test residuals in log space then converting)
y_pred_test_log = best_metrics['y_pred_test_log']
residuals_log = y_test_log - y_pred_test_log
residual_std_log = residuals_log.std()

# Calculate interval in log space, then convert to original scale
prediction_low_log = avatar_pred_log - 1.96 * residual_std_log
prediction_high_log = avatar_pred_log + 1.96 * residual_std_log

prediction_low = np.expm1(prediction_low_log)
prediction_high = np.expm1(prediction_high_log)

print(f"\n📊 95% Prediction Interval:")
print(f"   Lower Bound: ${max(0, prediction_low):,.0f}")
print(f"   Upper Bound: ${prediction_high:,.0f}")



STEP 8: PREDICTING OPENING REVENUE FOR 'AVATAR: FIRE AND ASH'

🎬 James Cameron Stats:
   Total Movies: 8
   Avg Total Gross: $372,014,938
   Avg Opening Revenue: $88,431,457
   Avg Budget: $256,000,000
   Avg TMDB Rating: 7.68

🏢 Walt Disney Studios Stats:
   Total Movies: 218
   Total Gross: $38,546,931,771
   Avg Opening Revenue: $69,577,768
   Market Share: 19.64%

📊 Predictions from all models:
   • XGBoost: $125,593,552
   • LightGBM: $129,777,721
   • CatBoost: $105,935,603
   • Random Forest: $132,071,878
   • Gradient Boosting: $166,117,328
   • Stacking Ensemble: $134,582,445

   🎯 Ensemble Average: $132,346,421

🎬 PREDICTION RESULTS FOR 'AVATAR: FIRE AND ASH'

🎯 Predicted Opening Revenue (Gradient Boosting): $166,117,328
🎯 Ensemble Average Prediction: $132,346,421

📊 Prediction Context:
   • Release Date: December 19, 2025 (Friday, Holiday Season)
   • Budget: $350,000,000
   • Director: James Cameron (Avg Opening: $88,431,457)
   • Distributor: Walt Disney Studios (Avg Open

### Step 9: Save Model and Results

This cell handles the persistence of the trained models and the generated results, ensuring reproducibility and the ability to deploy or further analyze the project outputs.

1.  **Best Model Saving:** The `best_model` (identified in Step 6) is saved to a file named `best_opening_revenue_model.pkl` using `joblib.dump`. This allows the model to be loaded and used later for inference without retraining.
2.  **All Models Saving:** A dictionary containing all individual and ensemble models trained during the process is saved as `all_models_ensemble.pkl`. This is useful for future experimentation or for building custom ensemble predictions.
3.  **Feature List Saving:** The list of `available_features` used to train the models is saved to `model_features.txt`. This is crucial for consistency, ensuring that any new data for prediction is preprocessed and ordered with the exact same features.
4.  **Prediction Results Saving:** A summary of the 'Avatar: Fire and Ash' prediction, including the best model's prediction, ensemble average, prediction interval bounds, and the best model's performance metrics, is saved to `avatar_prediction_results.csv`.
5.  **Model Comparison Saving:** The `comparison_df`, which details the performance of all models, is saved to `model_comparison_results.csv`.

These saved files represent the tangible outcomes of the modeling phase and are ready for deployment, sharing, or further analysis.

In [16]:
# ============================================================================
# STEP 9: SAVE MODEL AND RESULTS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 9: SAVING MODEL AND RESULTS")
print("=" * 80)

# Save the best model
import joblib
joblib.dump(best_model, 'best_opening_revenue_model.pkl')
print(f"✅ Best model ({best_model_name}) saved as 'best_opening_revenue_model.pkl'")

# Save all models for ensemble predictions
all_models = {name: res['model'] for name, res in results.items()}
joblib.dump(all_models, 'all_models_ensemble.pkl')
print("✅ All models saved as 'all_models_ensemble.pkl'")

# Save feature list
with open('model_features.txt', 'w') as f:
    f.write('\n'.join(available_features))
print("✅ Feature list saved as 'model_features.txt'")

# Save prediction results
results_dict = {
    'movie': 'Avatar: Fire and Ash',
    'best_model': best_model_name,
    'predicted_opening_revenue': avatar_prediction,
    'ensemble_avg_prediction': ensemble_avg,
    'prediction_lower_bound': max(0, prediction_low),
    'prediction_upper_bound': prediction_high,
    'model_r2_score': test_metrics['r2'],
    'model_mae': test_metrics['mae'],
    'model_rmse': test_metrics['rmse'],
    'model_mape': test_metrics['mape']
}

results_df = pd.DataFrame([results_dict])
results_df.to_csv('avatar_prediction_results.csv', index=False)
print("✅ Prediction results saved as 'avatar_prediction_results.csv'")

# Save model comparison
comparison_df.to_csv('model_comparison_results.csv', index=False)
print("✅ Model comparison saved as 'model_comparison_results.csv'")



STEP 9: SAVING MODEL AND RESULTS
✅ Best model (Gradient Boosting) saved as 'best_opening_revenue_model.pkl'
✅ All models saved as 'all_models_ensemble.pkl'
✅ Feature list saved as 'model_features.txt'
✅ Prediction results saved as 'avatar_prediction_results.csv'
✅ Model comparison saved as 'model_comparison_results.csv'


### Step 10: Create Visualizations

This cell generates several insightful plots to visually analyze the model's performance and present the prediction results.

1.  **Actual vs. Predicted Plot:** A scatter plot comparing the `y_test_orig` (actual values from the test set) against `y_pred_test` (predictions from the `best_model`). A red dashed line representing perfect prediction helps to visualize the model's accuracy and identify any systemic biases.
2.  **Residual Distribution:** A histogram showing the distribution of the residuals (errors) on the original scale (`y_test_orig - y_pred_test`). A healthy model typically has residuals centered around zero, often following a normal distribution.
3.  **Model Comparison Bar Chart:** A horizontal bar chart displaying the `Test R²` scores for all models compared. The `best_model` is highlighted to emphasize its superior performance.
4.  **Avatar Prediction Bar Chart:** A bar chart illustrating the predicted opening revenue for 'Avatar: Fire and Ash' from each individual model and the `ensemble_avg`. This helps visualize the consensus and variation across different models for this specific prediction.

All plots are carefully designed for clarity and saved together in a single image file named `model_analysis.png`.

In [17]:

# ============================================================================
# STEP 10: VISUALIZATION
# ============================================================================

print("\n" + "=" * 80)
print("STEP 10: CREATING VISUALIZATIONS")
print("=" * 80)

# 10.1 Actual vs Predicted Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Actual vs Predicted
ax1 = axes[0, 0]
ax1.scatter(y_test_orig, y_pred_test, alpha=0.5, edgecolors='black', linewidth=0.5)
ax1.plot([y_test_orig.min(), y_test_orig.max()], [y_test_orig.min(), y_test_orig.max()], 'r--', lw=2)
ax1.set_xlabel('Actual Opening Revenue')
ax1.set_ylabel('Predicted Opening Revenue')
ax1.set_title(f'Actual vs Predicted - {best_model_name}')
ax1.ticklabel_format(style='plain', axis='both')

# Plot 2: Residual Distribution
ax2 = axes[0, 1]
residuals_orig = y_test_orig - y_pred_test
ax2.hist(residuals_orig, bins=50, edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='r', linestyle='--', lw=2)
ax2.set_xlabel('Residuals (Actual - Predicted)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Residuals')

# Plot 3: Model Comparison Bar Chart
ax3 = axes[1, 0]
model_names = comparison_df['Model'].tolist()
test_r2_scores = comparison_df['Test R²'].tolist()
colors = ['green' if name == best_model_name else 'steelblue' for name in model_names]
bars = ax3.barh(range(len(model_names)), test_r2_scores, color=colors)
ax3.set_yticks(range(len(model_names)))
ax3.set_yticklabels(model_names)
ax3.set_xlabel('Test R² Score')
ax3.set_title('Model Comparison - Test R² Score')
ax3.invert_yaxis()
# Add value labels
for i, (bar, score) in enumerate(zip(bars, test_r2_scores)):
    ax3.text(score + 0.01, bar.get_y() + bar.get_height()/2, f'{score:.4f}', va='center')

# Plot 4: All Model Predictions for Avatar
ax4 = axes[1, 1]
pred_names = list(all_predictions.keys()) + ['Ensemble Avg']
pred_values = list(all_predictions.values()) + [ensemble_avg]
colors = ['orange' if name == best_model_name or name == 'Ensemble Avg' else 'steelblue' for name in pred_names]
bars = ax4.bar(range(len(pred_names)), pred_values, color=colors, edgecolor='black')
ax4.set_xticks(range(len(pred_names)))
ax4.set_xticklabels(pred_names, rotation=45, ha='right')
ax4.set_ylabel('Predicted Opening Revenue ($)')
ax4.set_title('Avatar: Fire and Ash - Predictions by Model')
ax4.ticklabel_format(style='plain', axis='y')

# Add value labels on bars
for bar, val in zip(bars, pred_values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2000000,
             f'${val/1e6:.0f}M', ha='center', va='bottom', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig('model_analysis.png', dpi=150, bbox_inches='tight')
plt.close()
print("✅ Model analysis plot saved as 'model_analysis.png'")




STEP 10: CREATING VISUALIZATIONS
✅ Model analysis plot saved as 'model_analysis.png'


### Final Summary

This cell provides a concise and comprehensive overview of the entire project's results, bringing together the key findings and the final prediction in an easily digestible format.

1.  **Prediction Summary Box:** A well-formatted text box presents the most important results:
    *   **Best Model:** Clearly states the name of the top-performing model.
    *   **Predicted Opening Revenue:** Shows the `best_model`'s prediction for 'Avatar: Fire and Ash'.
    *   **Ensemble Average:** Presents the average prediction across all models, often a more reliable estimate.
    *   **Model Performance:** Summarizes the `best_model`'s performance metrics (R² Score, MAE, RMSE, MAPE) on the original scale of the target variable.
    *   **95% Confidence Interval:** Provides the lower and upper bounds of the prediction interval for 'Avatar: Fire and Ash', indicating the range within which the true value is likely to fall.
    *   **Franchise Context:** Lists the opening revenues of previous 'Avatar' movies for comparative analysis.

2.  **Generated Files List:** A list of all output files created and saved throughout the notebook (`.pkl`, `.txt`, `.csv`, `.png`) is provided, making it easy to locate and access all project artifacts.

In [18]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("🎬 FINAL SUMMARY")
print("=" * 80)

print(f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    AVATAR: FIRE AND ASH - PREDICTION SUMMARY                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  � BEST MODEL: {best_model_name:<30}
║                                                                              ║
║  🎯 PREDICTED OPENING REVENUE: ${avatar_prediction:>15,.0f}
║  🎯 ENSEMBLE AVERAGE:          ${ensemble_avg:>15,.0f}
║                                                                              ║
║  📊 MODEL PERFORMANCE (with Log Transform):                                  ║
║     • R² Score:  {test_metrics['r2']:.4f}
║     • MAE:       ${test_metrics['mae']:>12,.0f}
║     • RMSE:      ${test_metrics['rmse']:>12,.0f}
║     • MAPE:      {test_metrics['mape']:.2f}%
║                                                                              ║
║  📈 95% CONFIDENCE INTERVAL:                                                 ║
║     • Lower: ${max(0, prediction_low):>15,.0f}
║     • Upper: ${prediction_high:>15,.0f}
║                                                                              ║
║  🎬 FRANCHISE CONTEXT:                                                       ║
║     • Avatar (2009):           $137,094,001 opening                          ║
║     • Avatar: Way of Water:    $197,681,686 opening                          ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

print("\n✅ Script completed successfully!")
print("📁 Generated files:")
print("   • best_opening_revenue_model.pkl - Best trained model")
print("   • all_models_ensemble.pkl - All models for ensemble")
print("   • model_features.txt - Feature list")
print("   • avatar_prediction_results.csv - Prediction results")
print("   • model_comparison_results.csv - Model comparison")
print("   • feature_importance.png - Feature importance chart")
print("   • model_analysis.png - Model analysis visualizations")



🎬 FINAL SUMMARY

╔══════════════════════════════════════════════════════════════════════════════╗
║                    AVATAR: FIRE AND ASH - PREDICTION SUMMARY                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  � BEST MODEL: Gradient Boosting                                      
║                                                                              ║
║  🎯 PREDICTED OPENING REVENUE: $    166,117,328                   
║  🎯 ENSEMBLE AVERAGE:          $    132,346,421                   
║                                                                              ║
║  📊 MODEL PERFORMANCE (with Log Transform):                                  ║
║     • R² Score:  0.7286                                       
║     • MAE:       $   8,350,308                                 
║     • RMSE:      $  20,702,441                                
║     • MA

## Final Comments and Insights

The project successfully built and evaluated several machine learning models to predict movie opening revenue, culminating in a robust prediction for 'Avatar: Fire and Ash'.

**Key Insights and Results:**

*   **Best Performing Model:** Gradient Boosting achieved the highest performance with a Test R² of 0.7286, a Test MAE of $8,350,308, a Test RMSE of $20,702,441, and a Test MAPE of 290.68%. This model showed good generalization capabilities, confirmed by 5-fold cross-validation (Mean CV R²: 0.7912).
*   **Prediction for 'Avatar: Fire and Ash':**
    *   **Best Model Prediction:** $166,117,328
    *   **Ensemble Average Prediction:** $132,346,421
    *   **95% Prediction Interval:** Between $30,732,872 and $897,897,413.
*   **Feature Importance:** Director's average opening revenue and distributor's average opening revenue were identified as the most influential features, highlighting the strong impact of past performance and industry relationships on a movie's box office success.
*   **Data Transformation:** Log transformation of the target variable was crucial in handling the skewed nature of revenue data, significantly improving model performance.
*   **Visualization:** Various plots (Actual vs. Predicted, Residual Distribution, Model Comparison, and Avatar Predictions) provide clear visual evidence of model performance and the specific prediction.

All detailed results, including the best model, feature lists, and visualizations, have been saved to files:
*   `best_opening_revenue_model.pkl`
*   `all_models_ensemble.pkl`
*   `model_features.txt`
*   `avatar_prediction_results.csv`
*   `model_comparison_results.csv`
*   `feature_importance.png`
*   `model_analysis.png`

This project demonstrates a complete end-to-end machine learning pipeline for revenue prediction, from data preprocessing and feature engineering to multi-model evaluation and specific-case prediction with uncertainty estimation.